In [ ]:
#%env TORCH_FABRIC_DISABLE_ATOMIC_SAVE=1
#%env PL_DISABLE_FSSPEC=1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!cd /content && git clone https://github.com/ntua-el21050/bd3lms.git


Cloning into 'bd3lms'...
remote: Enumerating objects: 750, done.
remote: Counting objects: 100% (219/219), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 750 (delta 195), reused 173 (delta 171), pack-reused 531 (from 1)
Receiving objects: 100% (750/750), 1.12 MiB | 23.85 MiB/s, done.
Resolving deltas: 100% (488/488), done.


In [ ]:
#!mkdir -p /content/drive/MyDrive/bd3lms_storage/{outputs,logs,watch_folder,sample_logs}


In [ ]:
#!rm -rf /content/bd3lms/outputs \
 #       /content/bd3lms/logs \
  #      /content/bd3lms/watch_folder \
   #     /content/bd3lms/sample_logs


In [ ]:
#!ln -s /content/drive/MyDrive/bd3lms_storage/outputs /content/bd3lms/outputs
#!ln -s /content/drive/MyDrive/bd3lms_storage/logs /content/bd3lms/logs
#!ln -s /content/drive/MyDrive/bd3lms_storage/watch_folder /content/bd3lms/watch_folder
#!ln -s /content/drive/MyDrive/bd3lms_storage/sample_logs /content/bd3lms/sample_logs


In [ ]:
!ls -l /content/bd3lms


total 380
drwxr-xr-x 9 root root   4096 Jan  1 19:17 configs
-rw-r--r-- 1 root root  33535 Jan  1 19:17 dataloader.py
-rw-r--r-- 1 root root  43433 Jan  1 19:17 diffusion.py
-rw-r--r-- 1 root root 225205 Jan  1 19:17 graphical_abstract.png
-rw-r--r-- 1 root root  11357 Jan  1 19:17 LICENSE
-rw-r--r-- 1 root root   7873 Jan  1 19:17 main.py
-rw-r--r-- 1 root root   8405 Jan  1 19:17 metrics.py
drwxr-xr-x 3 root root   4096 Jan  1 19:17 models
-rw-r--r-- 1 root root   2538 Jan  1 19:17 noise_schedule.py
-rw-r--r-- 1 root root   1449 Jan  1 19:17 push_to_hf.py
-rw-r--r-- 1 root root  10070 Jan  1 19:17 README.md
-rw-r--r-- 1 root root    363 Jan  1 19:17 requirements.txt
drwxr-xr-x 7 root root   4096 Jan  1 19:17 scripts
drwxr-xr-x 4 root root   4096 Jan  1 19:17 ssd-lm
-rw-r--r-- 1 root root   6855 Jan  1 19:17 utils.py


In [ ]:
!pip install -q \
    torchmetrics==1.6.2 \
    datasets==3.3.2 \
    einops==0.8.1 \
    fsspec==2024.2.0 \
    hydra-core==1.3.2 \
    lightning==2.5.0.post0 \
    omegaconf==2.3.0 \
    packaging==23.2 \
    pandas==2.2.1 \
    rich==13.7.1 \
    scikit-learn==1.5.1 \
    timm==0.9.16 \
    transformers==4.49.0 \
    matplotlib==3.10.0 \
    wandb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 931.6/931.6 kB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 148.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.7/240.7 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 148.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import sys
sys.path.append("/content")


In [ ]:
!cd /content/bd3lms && python main.py \
  mode=train \
  model=tiny \
  algo=ar \
  data=lm1b_wrap \
  model.length=128 \
  model.attn_backend=sdpa \
  block_size=1 \
  trainer.devices=1 \
  loader.global_batch_size=4 \
  loader.batch_size=4 \
  loader.eval_batch_size=4 \
  trainer.max_steps=800 \
  data.max_train_samples=500 \
  data.max_valid_samples=100 \
  data.max_test_samples=100 \
  training.nll_diagram=True


Seed set to 1
CONFIG
├── mode
│   └── train                                                                   
├── diffusion
│   └── absorbing_state                                                         
├── seed
│   └── 1                                                                       
├── block_size
│   └── 1                                                                       
├── data
│   └── max_samples: null                                                       
│       max_train_samples: 500                                                  
│       max_valid_samples: 100                                                  
│       max_test_samples: 100                                                   
│       train: lm1b                                                             
│       valid: lm1b                                                             
│       tokenizer_name_or_path: bert-base-uncased                               
│       cache_dir: /share/kulesh

In [ ]:
!mkdir -p /content/drive/MyDrive/bd3lms_storage
!rsync -av \
  /content/bd3lms/outputs/ \
  /content/drive/MyDrive/bd3lms_storage/outputs/


sending incremental file list
./
lm1b/
lm1b/2026.01.01/
lm1b/2026.01.01/191828/
lm1b/2026.01.01/191828/.hydra/
lm1b/2026.01.01/191828/checkpoints/
lm1b/2026.01.01/193144/
lm1b/2026.01.01/193144/.hydra/
lm1b/2026.01.01/193144/checkpoints/
lm1b/2026.01.01/194218/
lm1b/2026.01.01/194218/config_tree.txt
lm1b/2026.01.01/194218/main.log
lm1b/2026.01.01/194218/nll_diagram.png
lm1b/2026.01.01/194218/.hydra/
lm1b/2026.01.01/194218/.hydra/config.yaml
lm1b/2026.01.01/194218/.hydra/hydra.yaml
lm1b/2026.01.01/194218/.hydra/overrides.yaml
lm1b/2026.01.01/194218/checkpoints/
lm1b/2026.01.01/194218/checkpoints/16-500.ckpt
lm1b/2026.01.01/194218/checkpoints/33-1000.ckpt
lm1b/2026.01.01/194218/checkpoints/best.ckpt
lm1b/2026.01.01/194218/checkpoints/last.ckpt
lm1b/2026.01.01/210608/
lm1b/2026.01.01/210608/config_tree.txt
lm1b/2026.01.01/210608/main.log
lm1b/2026.01.01/210608/.hydra/
lm1b/2026.01.01/210608/.hydra/config.yaml
lm1b/2026.01.01/210608/.hydra/hydra.yaml
lm1b/2026.01.01/210608/.hydra/overrides

In [ ]:
#eval_bd3lm_last

!cd /content/bd3lms && python main.py \
  mode=ppl_eval \
  model=tiny \
  algo=ar \
  data=lm1b_wrap \
  model.length=128 \
  model.attn_backend=sdpa \
  block_size=1 \
  trainer.devices=1 \
  trainer.num_nodes=1 \
  loader.global_batch_size=4 \
  loader.batch_size=4 \
  loader.eval_batch_size=4 \
  eval.checkpoint_path=/content/drive/MyDrive/bd3lms_storage/outputs/lm1b/2026.01.01/ar_800/checkpoints/best.ckpt \
  data.max_valid_samples=100 \
  data.max_test_samples=100

Seed set to 1
CONFIG
├── mode
│   └── ppl_eval                                                                
├── diffusion
│   └── absorbing_state                                                         
├── seed
│   └── 1                                                                       
├── block_size
│   └── 1                                                                       
├── data
│   └── max_samples: null                                                       
│       max_train_samples: null                                                 
│       max_valid_samples: 100                                                  
│       max_test_samples: 100                                                   
│       train: lm1b                                                             
│       valid: lm1b                                                             
│       tokenizer_name_or_path: bert-base-uncased                               
│       cache_dir: /share/kulesh

In [ ]:
'''# ===== Step 0: Paths =====
REPO_DIR="/content/bd3lms"
TMP_OUTPUTS="/content/tmp_outputs"
DRIVE_OUTPUTS="/content/drive/MyDrive/bd3lms_storage/outputs"

# ===== Step 1: Backup outputs locally =====
!mkdir -p $TMP_OUTPUTS
!rsync -av --progress $REPO_DIR/outputs/ $TMP_OUTPUTS/

# ===== Step 2: Refresh repo =====
!cd $REPO_DIR && git fetch origin && git reset --hard origin/main && git pull

# ===== Step 3: Restore outputs =====
!rsync -av --progress $TMP_OUTPUTS/ $REPO_DIR/outputs/

# ===== Step 4: Clean temporary backup =====
!rm -rf $TMP_OUTPUTS

!echo "✅ Repo refreshed and outputs restored!"'''


sending incremental file list
./
lm1b/
lm1b/2026.01.01/
lm1b/2026.01.01/191828/
lm1b/2026.01.01/191828/config_tree.txt
         12,380 100%    0.00kB/s    0:00:00 (xfr#1, to-chk=9/14)
lm1b/2026.01.01/191828/main.log
            533 100%  520.51kB/s    0:00:00 (xfr#2, to-chk=8/14)
lm1b/2026.01.01/191828/nll_diagram.png
         63,439 100%   60.50MB/s    0:00:00 (xfr#3, to-chk=7/14)
lm1b/2026.01.01/191828/.hydra/
lm1b/2026.01.01/191828/.hydra/config.yaml
          3,374 100%    3.22MB/s    0:00:00 (xfr#4, to-chk=4/14)
lm1b/2026.01.01/191828/.hydra/hydra.yaml
          4,099 100%    3.91MB/s    0:00:00 (xfr#5, to-chk=3/14)
lm1b/2026.01.01/191828/.hydra/overrides.yaml
            351 100%  342.77kB/s    0:00:00 (xfr#6, to-chk=2/14)
lm1b/2026.01.01/191828/checkpoints/
lm1b/2026.01.01/191828/checkpoints/best.ckpt
    366,074,417 100%  453.40MB/s    0:00:00 (xfr#7, to-chk=1/14)
lm1b/2026.01.01/191828/checkpoints/last.ckpt
    366,074,417 100%  159.63MB/s    0:00:02 (xfr#8, to-chk=0/14)

sent